In [ ]:
import os
from PublicDataReader import TransactionPrice
service_key = os.getenv("PUBLIC_DATA_HUB")
api = TransactionPrice(service_key)

df = api.get_data(
    property_type="아파트",
    trade_type="매매",
    sigungu_code="11650",
    start_year_month="202401",
    end_year_month="202412",
)

# df.to_csv("아파트_매매_202501_202506.csv", index=False)
df

In [29]:
import PublicDataReader as pdr
sigungu_name = "강남구"
code = pdr.code_bdong()
code.loc[(code['시군구명'].str.contains(sigungu_name)) &
         (code['읍면동명']=='')]

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.8.1. 시행)


,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
1003,11,서울특별시,11680,강남구,1168000000,,,19880423,


In [2]:
import os
from PublicDataReader import Kosis


# KOSIS 공유서비스 Open API 사용자 인증키
service_key = os.getenv("KOSIS")

# 인스턴스 생성하기
api = Kosis(service_key)

In [10]:
df1 = api.get_data(
    "KOSIS통합검색",
    searchNm="아파트거래통계",
    )
df1.head(1)

,기관ID,기관명,통계표ID,통계표명,조사ID,조사명,KOSIS목록구분,KOSIS통계표위치,통계표위치,통계표주요내용,수록기간시작일,수록기간종료일,통계표주석,추천통계표여부,KOSIS목록URL,KOSIS통계표URL,검색결과건수,검색어명
0,408,한국부동산원,DT_KAB_11672_S25,아파트 전세 실거래 평균가격,2009045,공동주택실거래가격지수,MT_ZTITLE,주거 > 공동주택실거래가격지수,I1 > I1_1,행정구역별 가격 광역시 부산 제주 광주 충남 전북 서울 서북권 도 충북 전남 경남 ...,2014,2025,*공동주택 실거래가격지수 중 전세 실거래가격지수는 확정일자를 기초로 신규계약 및 재...,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=408...,372,아파트거래통계


In [ ]:
df2 = api.get_data(
    "통계표설명"
    orgId = "408",
    tblId = "DT_KAB_11672_S25",
    metaItm = "ALL",
    )
df2.to_csv("아파트거래통계.csv")

In [15]:
# 아파트거래량 통계 만들기 - 추천 방법
import os
import pandas as pd
from PublicDataReader import TransactionPrice
import matplotlib.pyplot as plt

# 서비스키 설정
service_key = os.getenv("PUBLIC_DATA_HUB")
api = TransactionPrice(service_key)

# 서울 주요 구 데이터 가져오기
regions = {
    "강남구": "11680",
    "서초구": "11650", 
    "송파구": "11710",
    "강동구": "11740",
    "마포구": "11560"
}

all_data = []

for region_name, sigungu_code in regions.items():
    print(f"{region_name} 데이터 수집 중...")
    try:
        region_df = api.get_data(
            property_type="아파트",
            trade_type="매매",
            sigungu_code=sigungu_code,
            start_year_month="202401",
            end_year_month="202412",
        )
        if region_df is not None and len(region_df) > 0:
            region_df['지역'] = region_name
            all_data.append(region_df)
            print(f"✅ {region_name}: {len(region_df)}건")
        else:
            print(f"❌ {region_name}: 데이터 없음")
    except Exception as e:
        print(f"❌ {region_name}: {e}")

# 데이터 통합
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\n📊 총 {len(combined_df)}건의 거래 데이터 수집 완료")
    
    # 거래량 통계 집계
    print("\n=== 월별 거래량 통계 ===")
    monthly_stats = combined_df.groupby(['계약년월', '지역']).size().reset_index(name='거래건수')
    monthly_pivot = monthly_stats.pivot(index='계약년월', columns='지역', values='거래건수').fillna(0)
    
    print(monthly_pivot)
    
    # 지역별 총 거래량
    print("\n=== 지역별 총 거래량 ===")
    region_stats = combined_df.groupby('지역').agg({
        '시군구': 'count',
        '거래금액': lambda x: x.str.replace(',', '').astype(int).mean(),
        '전용면적': 'mean'
    }).rename(columns={'시군구': '거래건수', '거래금액': '평균거래금액'})
    
    print(region_stats)
    
    # 시각화
    plt.figure(figsize=(12, 6))
    monthly_pivot.plot(kind='bar', stacked=True)
    plt.title('월별 지역별 아파트 거래량')
    plt.xlabel('계약년월')
    plt.ylabel('거래건수')
    plt.legend(title='지역', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 데이터 저장
    combined_df.to_csv('아파트_거래량_통계.csv', index=False)
    monthly_pivot.to_csv('월별_지역별_거래량.csv')
    
    print("\n✅ 데이터 저장 완료:")
    print("- 아파트_거래량_통계.csv: 전체 거래 데이터")
    print("- 월별_지역별_거래량.csv: 월별 지역별 거래량 통계")
    
else:
    print("❌ 데이터 수집에 실패했습니다.")


강남구 데이터 수집 중...
✅ 강남구: 3691건
서초구 데이터 수집 중...
✅ 서초구: 3060건
송파구 데이터 수집 중...
✅ 송파구: 4315건
강동구 데이터 수집 중...
✅ 강동구: 3429건
마포구 데이터 수집 중...
✅ 마포구: 2961건

📊 총 17456건의 거래 데이터 수집 완료

=== 월별 거래량 통계 ===


KeyError: '계약년월'

In [16]:
# KOSIS API로 아파트거래통계 가져오기 - 보완 방법
import os
import pandas as pd
from PublicDataReader import Kosis

# KOSIS API 키 설정
service_key = os.getenv("KOSIS")
api = Kosis(service_key)

print("=== KOSIS API를 사용한 아파트거래통계 ===")

# 1. 아파트거래통계 관련 통계표 검색
print("1. 아파트거래통계 관련 통계표 검색 중...")
try:
    search_df = api.get_data(
        "KOSIS통합검색",
        searchNm="아파트거래통계",
    )
    
    if search_df is not None and len(search_df) > 0:
        print(f"✅ 검색 결과: {len(search_df)}개 통계표 발견")
        print("\n주요 통계표:")
        print(search_df[['기관명', '통계표명', '통계표ID']].head())
        
        # 가장 관련성이 높은 통계표 선택
        apartment_stats = search_df[search_df['통계표명'].str.contains('아파트|거래', na=False)]
        if len(apartment_stats) > 0:
            selected_table = apartment_stats.iloc[0]
            print(f"\n선택된 통계표: {selected_table['통계표명']}")
            print(f"기관: {selected_table['기관명']}")
            print(f"통계표ID: {selected_table['통계표ID']}")
            
            # 통계표 메타데이터 조회
            print("\n2. 통계표 메타데이터 조회 중...")
            try:
                meta_df = api.get_data(
                    "통계표설명",
                    "분류항목",
                    orgId=selected_table['기관ID'],
                    tblId=selected_table['통계표ID'],
                    metaItm="ALL",
                )
                
                if meta_df is not None and len(meta_df) > 0:
                    print(f"✅ 메타데이터 조회 성공: {len(meta_df)}개 항목")
                    print(meta_df[['분류명', '분류값명']].head(10))
                    
                    # 실제 통계 데이터 조회
                    print("\n3. 실제 통계 데이터 조회 중...")
                    try:
                        data_df = api.get_data(
                            "통계자료",
                            orgId=selected_table['기관ID'],
                            tblId=selected_table['통계표ID'],
                            startPrdDe="202401",
                            endPrdDe="202412",
                        )
                        
                        if data_df is not None and len(data_df) > 0:
                            print(f"✅ 통계 데이터 조회 성공: {len(data_df)}개 데이터")
                            print("\n데이터 미리보기:")
                            print(data_df.head())
                            
                            # 데이터 저장
                            data_df.to_csv('KOSIS_아파트거래통계.csv', index=False)
                            print("\n✅ KOSIS_아파트거래통계.csv 저장 완료")
                            
                        else:
                            print("❌ 통계 데이터 조회 실패 - 데이터 없음")
                            
                    except Exception as e:
                        print(f"❌ 통계 데이터 조회 에러: {e}")
                        
                else:
                    print("❌ 메타데이터 조회 실패")
                    
            except Exception as e:
                print(f"❌ 메타데이터 조회 에러: {e}")
        else:
            print("❌ 아파트거래 관련 통계표를 찾을 수 없습니다.")
    else:
        print("❌ 검색 결과 없음")
        
except Exception as e:
    print(f"❌ 검색 에러: {e}")

# 4. 대안: 국토교통부 통계 직접 조회
print("\n=== 대안: 국토교통부 통계 직접 조회 ===")
try:
    # 국토교통부(408) 아파트거래통계 (DT_KAB_11672_S25)
    molit_df = api.get_data(
        "통계자료",
        orgId="408",
        tblId="DT_KAB_11672_S25",
        startPrdDe="202401",
        endPrdDe="202412",
    )
    
    if molit_df is not None and len(molit_df) > 0:
        print(f"✅ 국토교통부 아파트거래통계 조회 성공: {len(molit_df)}개 데이터")
        print("\n데이터 미리보기:")
        print(molit_df.head())
        
        # 데이터 저장
        molit_df.to_csv('국토교통부_아파트거래통계.csv', index=False)
        print("\n✅ 국토교통부_아파트거래통계.csv 저장 완료")
        
    else:
        print("❌ 국토교통부 데이터 조회 실패")
        
except Exception as e:
    print(f"❌ 국토교통부 데이터 조회 에러: {e}")

print("\n=== 결론 ===")
print("KOSIS API를 통해 공식 통계 데이터를 가져올 수 있습니다.")
print("- 장점: 공식 통계, 신뢰성 높음")
print("- 단점: 업데이트 주기가 상대적으로 길음")


=== KOSIS API를 사용한 아파트거래통계 ===
1. 아파트거래통계 관련 통계표 검색 중...
✅ 검색 결과: 20개 통계표 발견

주요 통계표:
      기관명                 통계표명             통계표ID
0  한국부동산원      아파트 전세 실거래 평균가격  DT_KAB_11672_S25
1  한국부동산원      아파트 전세 실거래 중위가격  DT_KAB_11672_S26
2  한국부동산원   아파트 규모별 전세 실거래가격지수  DT_KAB_11672_S24
3  한국부동산원       아파트 전세 실거래가격지수  DT_KAB_11672_S23
4  한국부동산원  아파트 규모별 전세 실거래 중위가격  DT_KAB_11672_S32

선택된 통계표: 아파트 전세 실거래 평균가격
기관: 한국부동산원
통계표ID: DT_KAB_11672_S25

2. 통계표 메타데이터 조회 중...
✅ 메타데이터 조회 성공: 29개 항목
     분류명 분류값명
0     항목   가격
1  행정구역별   전국
2  행정구역별  수도권
3  행정구역별   지방
4  행정구역별   서울
5  행정구역별   서울
6  행정구역별  도심권
7  행정구역별  동북권
8  행정구역별  동남권
9  행정구역별  서북권

3. 실제 통계 데이터 조회 중...
필수요청변수값이 누락되었습니다.
❌ 통계 데이터 조회 실패 - 데이터 없음

=== 대안: 국토교통부 통계 직접 조회 ===
필수요청변수값이 누락되었습니다.
❌ 국토교통부 데이터 조회 실패

=== 결론 ===
KOSIS API를 통해 공식 통계 데이터를 가져올 수 있습니다.
- 장점: 공식 통계, 신뢰성 높음
- 단점: 업데이트 주기가 상대적으로 길음


In [18]:
# Reb API 문제 해결 방법
import os
import pandas as pd
from PublicDataReader import Reb

print("=== Reb API 문제 해결 시도 ===")

# 1. 환경 변수 확인
print("1. 환경 변수 확인:")
public_data_key = os.getenv("PUBLIC_DATA_HUB")
korea_reb_key = os.getenv("KOREA_Reb")

print(f"PUBLIC_DATA_HUB 키 존재: {public_data_key is not None}")
print(f"KOREA_Reb 키 존재: {korea_reb_key is not None}")

if public_data_key:
    print(f"PUBLIC_DATA_HUB 키 길이: {len(public_data_key)}")
if korea_reb_key:
    print(f"KOREA_Reb 키 길이: {len(korea_reb_key)}")

# 2. 올바른 API 키로 재시도
print("\n2. 올바른 API 키로 재시도:")
service_key = public_data_key or korea_reb_key

if service_key:
    api = Reb(service_key)
    
    print("사용 가능한 서비스:")
    for service_name in api.meta_dict.keys():
        print(f"- {service_name}")
    
    # 3. 부동산거래 서비스 상세 정보 확인
    print("\n3. 부동산거래 서비스 상세 정보:")
    if "부동산거래" in api.meta_dict:
        categories = api.meta_dict["부동산거래"]
        print(f"사용 가능한 카테고리: {len(categories)}개")
        
        for category, info in list(categories.items())[:5]:  # 처음 5개만 출력
            print(f"- {category}: {info.get('url', 'URL 없음')}")
    
    # 4. 다른 서비스로 테스트
    print("\n4. 다른 서비스로 테스트:")
    test_services = ["분양정보", "청약경쟁률", "공동주택단지정보"]
    
    for service_name in test_services:
        if service_name in api.meta_dict:
            print(f"\n{service_name} 서비스 테스트:")
            categories = list(api.meta_dict[service_name].keys())
            first_category = categories[0]
            print(f"첫 번째 카테고리: {first_category}")
            
            try:
                result = api.get_data(service_name, first_category)
                print(f"결과: {type(result)} - {result}")
                
                if result is not None:
                    print(f"✅ {service_name} 서비스 작동 확인!")
                    break
                else:
                    print(f"❌ {service_name} 서비스 None 반환")
                    
            except Exception as e:
                print(f"❌ {service_name} 서비스 에러: {e}")
    
    # 5. 직접 API 호출 확인
    print("\n5. 직접 API 호출 확인:")
    try:
        # 가장 간단한 방법으로 시도
        result = api.get_data("부동산거래", "거래건수", verbose=True)
        print(f"직접 호출 결과: {type(result)} - {result}")
        
        if result is not None:
            print("✅ 직접 호출 성공!")
        else:
            print("❌ 직접 호출도 None 반환")
            
    except Exception as e:
        print(f"❌ 직접 호출 에러: {e}")
        
    # 6. 대안 제안
    print("\n=== 대안 제안 ===")
    print("Reb API가 현재 제대로 작동하지 않는 경우:")
    print("1. TransactionPrice API 사용 (개별 거래 데이터)")
    print("2. KOSIS API 사용 (공식 통계 데이터)")
    print("3. 다른 공공 데이터 포털 API 사용")
    print("4. 웹 스크래핑 (법적 검토 필요)")
    
else:
    print("❌ 사용할 수 있는 API 키가 없습니다.")
    print("환경 변수를 확인해주세요:")
    print("- PUBLIC_DATA_HUB")
    print("- KOREA_Reb")


=== Reb API 문제 해결 시도 ===
1. 환경 변수 확인:
PUBLIC_DATA_HUB 키 존재: True
KOREA_Reb 키 존재: True
PUBLIC_DATA_HUB 키 길이: 94
KOREA_Reb 키 길이: 32

2. 올바른 API 키로 재시도:
사용 가능한 서비스:
- 분양정보
- 청약경쟁률
- 청약당첨정보
- 공동주택단지정보
- 공동주택실거래가격지수
- 주택
- 주간아파트
- 오피스텔
- 상업용임대
- 월세가격
- 부동산거래
- 지가변동률
- 연령대별부동산거래
- 녹색건축인증현황

3. 부동산거래 서비스 상세 정보:
사용 가능한 카테고리: 44개
- 거래면적: https://api.odcloud.kr/api/RealEstateTradingSvc/v1/getRealEstateTradingArea
- 거주지별거래면적: https://api.odcloud.kr/api/RealEstateTradingSvc/v1/getRealEstateTradingAreaResidence
- 거래건수: https://api.odcloud.kr/api/RealEstateTradingSvc/v1/getRealEstateTradingCount
- 거주지별거래건수: https://api.odcloud.kr/api/RealEstateTradingSvc/v1/getRealEstateTradingCountResidence
- 건물유형별거래면적: https://api.odcloud.kr/api/RealEstateTradingSvc/v1/getRealEstateTradingAreaBuildType

4. 다른 서비스로 테스트:

분양정보 서비스 테스트:
첫 번째 카테고리: 아파트
결과: <class 'pandas.core.frame.DataFrame'> -         BSNS_MBY_NM CNSTRCT_ENTRPS_NM CNTRCT_CNCLS_BGNDE CNTRCT_CNCLS_ENDDE  \
0     두류야외음악당지역주택조합           동부건설(주)        